### Today we are going to perform the simple classification of the amazon reviews' sentiment.

### Please, download the dataset amazon_baby.csv.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.linear_model import LogisticRegression
from google.colab import drive
def remove_punctuation(text):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)
drive.mount('/content/drive')
baby_df = pd.read_csv('/content/drive/MyDrive/dataScience/DisneylandReviews.csv', encoding = 'latin-1')
baby_df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Review_ID,Rating,Year_Month,Reviewer_Location,Review_Text,Branch
0,670772142,4,2019-4,Australia,If you've ever been to Disneyland anywhere you...,Disneyland_HongKong
1,670682799,4,2019-5,Philippines,Its been a while since d last time we visit HK...,Disneyland_HongKong
2,670623270,4,2019-4,United Arab Emirates,Thanks God it wasn t too hot or too humid wh...,Disneyland_HongKong
3,670607911,4,2019-4,Australia,HK Disneyland is a great compact park. Unfortu...,Disneyland_HongKong
4,670607296,4,2019-4,United Kingdom,"the location is not in the city, took around 1...",Disneyland_HongKong


In [10]:
baby_df.rename(columns = {'Reviewer_Location' : 'location', 'Review_Text' : 'review'}, inplace = True )

## Exercise 1 (data preparation)
a) Remove punctuation from reviews using the given function.   
b) Replace all missing (nan) revies with empty "" string.  
c) Drop all the entries with rating = 3, as they have neutral sentiment.   
d) Set all positive ($\geq$4) ratings to 1 and negative($\leq$2) to -1.

In [11]:
#b)
baby_df["review"] = baby_df["review"].fillna("")

#short test:
baby_df["review"][38] == baby_df["review"][38]

True

In [12]:
#a)
baby_df["review"] = baby_df["review"].apply(remove_punctuation)


#short test:
baby_df["review"][4] == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'
remove_punctuation(baby_df["review"][4]) == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'

False

In [15]:
#c)
baby_df = baby_df.drop(baby_df[baby_df.Rating == 3].index)

#short test:
sum(baby_df["Rating"] == 3)

0

In [17]:
print(baby_df["Rating"][:100])

0      4
1      4
2      4
3      4
4      4
      ..
114    5
115    4
116    5
117    4
118    4
Name: Rating, Length: 100, dtype: int64


In [18]:
#d)
baby_df.loc[baby_df["Rating"] <= 2, "Rating"] = -1
baby_df.loc[baby_df["Rating"] >= 4, "Rating"] = 1


#short test:
sum(baby_df["Rating"]**2 != 1)

0

In [19]:
print(baby_df["Rating"][:1000])

0       1
1       1
2       1
3       1
4       1
       ..
1147    1
1148    1
1149    1
1151    1
1152    1
Name: Rating, Length: 1000, dtype: int64


## CountVectorizer
In order to analyze strings, we need to assign them numerical values. We will use one of the simplest string representation, which transforms strings into the $n$ dimensional vectors. The number of dimensions will be the size of our dictionary, and then the values of the vector will represent the number of appereances of the given word in the sentence.

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
reviews_train_example = ["We like apples",
                   "We hate oranges",
                   "I adore bananas",
                   "We like like apples and oranges",
                   "They dislike bananas"]

X_train_example = vectorizer.fit_transform(reviews_train_example)

#print(vectorizer.get_feature_names())
print(X_train_example.todense())



[[0 0 1 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 1 0 1]
 [1 0 0 1 0 0 0 0 0 0]
 [0 1 1 0 0 0 2 1 0 1]
 [0 0 0 1 1 0 0 0 1 0]]


In [21]:
reviews_test_example = ["They like bananas",
                   "We hate oranges bananas and apples",
                   "We love bananas"] #New word!

X_test_example = vectorizer.transform(reviews_test_example)

print(X_test_example.todense())

[[0 0 0 1 0 0 1 0 1 0]
 [0 1 1 1 0 1 0 1 0 1]
 [0 0 0 1 0 0 0 0 0 1]]


We should acknowledge few facts. Firstly, CountVectorizer does not take order into account. Secondly, it ignores one-letter words (this can be changed during initialization). Finally, for test values, CountVectorizer ignores words which are not in it's dictionary.

## Exercise 2
a) Split dataset into training and test sets.     
b) Transform reviews into vectors using CountVectorizer.

In [22]:
#a)
from sklearn.model_selection import train_test_split


X = baby_df["review"][:5000]
y = baby_df["Rating"][:5000]

x_train, x_test, ra_train, ra_test = train_test_split(X, y, test_size=0.3, random_state=44)


In [23]:
#b)

re_train = vectorizer.fit_transform(x_train)
re_test = vectorizer.transform(x_test)




## Exercise 3
a) Train LogisticRegression model on training data (reviews processed with CountVectorizer, ratings as they were).   
b) Print 10 most positive and 10 most negative words.

In [24]:
print(baby_df["Rating"][:10])

0     1
1     1
2     1
3     1
4     1
6     1
8    -1
9     1
10    1
11    1
Name: Rating, dtype: int64


In [25]:
#a)
model = LogisticRegression(max_iter = 8000)
model.fit(re_train, ra_train)




LogisticRegression(max_iter=8000)

In [26]:
#b)
coefs, words = zip(*sorted(zip(model.coef_[0], vectorizer.get_feature_names_out())))
print('most positive words: ',tuple(reversed(words[-11:-1])))
print('most negative words: ',words[0:10])

#hint: model.coef_, vectorizer.get_feature_names()

most positive words:  ('great', 'clean', 'enjoyed', 'parade', 'though', 'however', 'loved', 'must', 'can', 'different')
most negative words:  ('disappointment', 'money', 'disappointing', 'closed', 'then', 'sun', 'disappointed', 'nothing', 'rude', 'few')


## Exercise 4
a) Predict the sentiment of test data reviews.   
b) Predict the sentiment of test data reviews in terms of probability.   
c) Find five most positive and most negative reviews.   
d) Calculate the accuracy of predictions.

In [27]:
#a)
sentiment_pred = model.predict(re_test)

In [28]:
#b)
sentiment_proba = model.predict_proba(re_test)
#hint: model.predict_proba()

In [29]:
#c)
pos_proba = sentiment_proba[:,1] #probability of a review being positive
neg_proba = sentiment_proba[:,0] #probability of a review being negative

proba_pos, re_pos = zip(*sorted(zip(pos_proba, x_test)))
proba_neg, re_neg = zip(*sorted(zip(neg_proba, x_test)))

print('Most positive reviews:\n')
for i in range(1, 6):
    print(f'{i}. {re_pos[-i][:500]}...')
    print('Probability =', proba_pos[-i], '\n')

print('\nMost negative reviews:\n')
for i in range(1, 6):
    print(f'{i}. {re_neg[-i][:500]}...')
    print('Probability =', proba_neg[-i], '\n')

#hint: use the results of b)

Most positive reviews:

1. Located in Lantau Island Hongkong Disneyland has always been dubbed as one of the most magical places on earth Yes it is truly magical as you will be seeing a lot of Disney characters you might have watched and sometimes your favorites   from Princesses to figure character and more You will be able to meet your favorite Disney characters watch live shows and parades and shop and dine at all themed areas of the Hong Kong Disneyland park HK Disneyland is accesible both via train car and buses Maki...
Probability = 1.0 

2. We have been to Disneyland in LA two times and Tokyo Disneyland Disney Sea Walking through the gates at HK Disneyland felt like going back to LA This is because the park is owned by Disney and HK Government We loved the quietness of HK Disneyland in the morning most things in HK dont open until 10am so to walk in with very few people in front of us was great Toy Storyland doesnt open until 11am We purchased our tickets online from Australia a

In [30]:
#d)
from sklearn.metrics import mean_squared_error

print("Train mean square error: ", mean_squared_error(ra_train, model.predict(re_train)))
print("Test mean square error: ", mean_squared_error(ra_test, sentiment_pred))
print("Coefficient of determination:", model.score(re_train, ra_train))

Train mean square error:  0.0
Test mean square error:  0.184
Coefficient of determination: 1.0


In [31]:
from sklearn.metrics import accuracy_score

print("Accuracy score: ", accuracy_score(ra_test, sentiment_pred))

Accuracy score:  0.954


## Exercise 5
In this exercise we will limit the dictionary of CountVectorizer to the set of significant words, defined below.


a) Redo exercises 2-5 using limited dictionary.   
b) Check the impact of all the words from the dictionary.   
c) Compare accuracy of predictions and the time of evaluation.

In [37]:
significant_words = ['great', 'clean', 'enjoyed', 'parade', 'though', 'however', 'loved', 'must', 'can', 'different', 'disappointment', 'money', 'disappointing', 'closed', 'then', 'sun', 'disappointed', 'nothing', 'rude', 'few']

In [38]:
#a)
vectorizer = CountVectorizer(vocabulary=significant_words)
model = LogisticRegression()

X_train_lim = vectorizer.fit_transform(x_train)
X_test_lim = vectorizer.transform(x_test)

model.fit(X_train_lim, ra_train)

coefs, words = zip(*sorted(zip(model.coef_[0], vectorizer.get_feature_names_out())))
print('most positive words: ',tuple(reversed(words[-11:-1])))
print('most negative words: ',words[0:10])

most positive words:  ('enjoyed', 'loved', 'great', 'must', 'though', 'clean', 'however', 'different', 'can', 'few')
most negative words:  ('rude', 'disappointment', 'disappointing', 'nothing', 'money', 'disappointed', 'closed', 'sun', 'then', 'few')


In [39]:
sentiment_pred = model.predict(X_test_lim)
sentiment_proba = model.predict_proba(X_test_lim)

pos_proba = sentiment_proba[:,1] #probability of a review being positive
neg_proba = sentiment_proba[:,0] #probability of a review being negative

proba_pos, re_pos = zip(*sorted(zip(pos_proba, x_test)))
proba_neg, re_neg = zip(*sorted(zip(neg_proba, x_test)))

print('Most positive reviews:\n')
for i in range(1, 6):
    print(f'{i}. {re_pos[-i][:500]}...')
    print('Probability =', proba_pos[-i], '\n')

print('\nMost negative reviews:\n')
for i in range(1, 6):
    print(f'{i}. {re_neg[-i][:500]}...')
    print('Probability =', proba_neg[-i], '\n')

Most positive reviews:

1. We lucked out as the day we decided to visit the park the weather was gorgeous I picked up a one day pass for everyone online so it was one less line to deal with at the parkGetting there the hotel staff recommended we just take a taxi as prices seem to be under control and it was a lot less hassle We took their advice and getting there involved zero stress or guesswork From Kowloon to the park was just under 200 HKDWe were dropped off very close to the main gate and while making your way to the...
Probability = 0.9999992856996549 

2. I came here expecting it to be rammed with people and even though there were a lot the park seemed to cope pretty well with the mass volume of visitors We caught the MTR to Sunny Bay and then caught the train   this was quite magical as a Disney themed train takes you to the theme park Trains were very frequent so no waiting around  Queues were not too bad I think the longest was 35 mins which is acceptable in summer holidays  

In [35]:
#b)
from tabulate import tabulate

table = [[words[i], coefs[i]] for i in range(0,20)]
print(tabulate(table, headers=["Word", "Coefficient"]))

Word            Coefficient
------------  -------------
disappointed     -1.32619
money            -1.17545
return           -1.11009
waste            -1.06827
broke            -0.780836
even             -0.45329
old              -0.251822
work             -0.172755
little           -0.143938
would            -0.111905
product           0
well              0.0802503
less              0.17773
car               0.327058
loves             0.396401
love              0.635125
able              0.732521
easy              0.973888
great             1.11673
perfect           1.29212


In [36]:
#c)

print("Train mean square error: ", mean_squared_error(ra_train, model.predict(X_train_lim)))
print("Test mean square error: ", mean_squared_error(ra_test, sentiment_pred))
print("Coefficient of determination:", model.score(X_train_lim, ra_train))
print("\nAccuracy score: ", accuracy_score(ra_test, sentiment_pred))
#hint: %time, %timeit

Train mean square error:  0.19542857142857142
Test mean square error:  0.248
Coefficient of determination: 0.9511428571428572

Accuracy score:  0.938


# Which Disneyland should we visit?
I am calculating sum of ratings in each of three Disneylands in this dataset and dividing it by the number of reviews (higher is better)

In [95]:
s = baby_df.groupby('Branch')['Rating'].sum().reset_index()
entries = baby_df['Branch'].value_counts()
s.iloc[1], s.iloc[2] = s.iloc[2].copy(), s.iloc[1].copy()
m = []

for x in range(3):
  m.append((s['Branch'][x], s['Rating'][x]/entries[x]))

best = max(m, key = lambda x: x[1])
worst = min(m, key = lambda x: x[1])
print('best: ', best)
print('worst: ', worst)


best:  ('Disneyland_HongKong', 0.8769230769230769)
worst:  ('Disneyland_Paris', 0.6757599376461418)
